# License

MIT License

Copyright (c) 2020 computational-sediment-hyd

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# import module

In [1]:
import numpy as np

# calculate a temporary velocity

In [2]:
def calupre(p,u,v,w,Vdir,Vbound,alphaU,dx,dy,dz,nu,axis):
# x: axis = 0, y: axis = 1, z: axis = 2
    
    zero = float(0.0)
    V0 = np.copy(Vdir)
    
    nx, ny, nz = Vdir.shape
    
    if axis == 0:
        nx -= 1
    elif axis == 1:
        ny -= 1
    elif axis == 2:
        nz -= 1
    
    ac  = np.zeros_like(Vdir)
    axp = np.zeros_like(Vdir)
    axm = np.zeros_like(Vdir)
    ayp = np.zeros_like(Vdir)
    aym = np.zeros_like(Vdir)
    azp = np.zeros_like(Vdir)
    azm = np.zeros_like(Vdir)
    b   = np.zeros_like(Vdir)
    
    act  = np.zeros_like(Vdir)
    
    if axis == 0:
        xs, ys, zs = 1, 0, 0
    elif axis == 1:
        xs, ys, zs = 0, 1, 0
    elif axis == 2:
        xs, ys, zs = 0, 0, 1
        
    if axis == 0:
        xmb, xpb, ymb, ypb, zmb, zpb = -1, -1, 0, ny-1, 0, nz-1
    elif axis == 1:
        xmb, xpb, ymb, ypb, zmb, zpb = 0, nx-1, -1, -1, 0, nz-1
    elif axis == 2:
        xmb, xpb, ymb, ypb, zmb, zpb = 0, nx-1, 0, ny-1, -1, -1
    
    for i in range(xs, nx):
        for j in range(ys, ny):
            for k in range(zs, nz):
                c = (i,j,k)
                
                if axis == 0:
                    uhxp = 0.5*u[i,j,k]   + 0.5*u[i+1,j,k]
                    uhxm = 0.5*u[i,j,k]   + 0.5*u[i-1,j,k]
                    vhyp = 0.5*v[i,j+1,k] + 0.5*v[i-1,j+1,k]
                    vhym = 0.5*v[i,j  ,k] + 0.5*v[i-1,j  ,k]
                    whzp = 0.5*w[i,j,k+1] + 0.5*w[i-1,j,k+1]
                    whzm = 0.5*w[i,j  ,k] + 0.5*w[i-1,j,k]
                elif axis == 1:
                    uhxp = 0.5*u[i+1,j,k] + 0.5*u[i+1,j-1,k]
                    uhxm = 0.5*u[i,j,k]   + 0.5*u[i,j-1,k]
                    vhyp = 0.5*v[i,j,k]   + 0.5*v[i,j+1,k]
                    vhym = 0.5*v[i,j,k]   + 0.5*v[i,j-1,k]
                    whzp = 0.5*w[i,j,k+1] + 0.5*w[i,j-1,k+1]
                    whzm = 0.5*w[i,j  ,k] + 0.5*w[i,j-1,k]
                elif axis == 2:
                    uhxp = 0.5*u[i+1,j,k] + 0.5*u[i+1,j,k-1]
                    uhxm = 0.5*u[i,j,k]   + 0.5*u[i,j,k-1]
                    vhyp = 0.5*v[i,j+1,k] + 0.5*v[i,j+1,k-1]
                    vhym = 0.5*v[i,j,k] + 0.5*v[i,j,k-1]
                    whzp = 0.5*w[i,j,k] + 0.5*w[i,j,k+1]
                    whzm = 0.5*w[i,j,k] + 0.5*w[i,j,k-1]
                
                axp[c] = (max([-uhxp,0.0]) + nu/dx)*dy*dz 
                axm[c] = (max([ uhxm,0.0]) + nu/dx)*dy*dz 
                ayp[c] = (max([-vhyp,0.0]) + nu/dy)*dx*dz 
                aym[c] = (max([ vhym,0.0]) + nu/dy)*dx*dz 
                azp[c] = (max([-whzp,0.0]) + nu/dz)*dx*dy 
                azm[c] = (max([ whzm,0.0]) + nu/dz)*dx*dy 
                
                ac[c] = axp[c]+axm[c]+ayp[c]+aym[c]+azp[c]+azm[c]
                
                if axis == 0:
                    b[c] = -dy*dz*( p[i,j,k] - p[i-1,j,k] ) + (1.0 - alphaU)/alphaU * ac[c] * V0[c]
                elif axis == 1:
                    b[c] = -dx*dz*( p[i,j,k] - p[i,j-1,k] ) + (1.0 - alphaU)/alphaU * ac[c] * V0[c]
                elif axis == 2:
                    b[c] = -dx*dy*( p[i,j,k] - p[i,j,k-1] ) + (1.0 - alphaU)/alphaU * ac[c] * V0[c]
                    
                ac[c] /= alphaU
                act[c] = ac[c]
                
                if i == xmb :
                    ac[c] += axm[c]
                elif i == xpb :
                    ac[c] += axp[c]
                
                if j == ymb :
                    ac[c] += aym[c]
                elif j == ypb :
                    ac[c] += ayp[c]
                    
                if k == zmb :
                    ac[c] += azm[c]
                elif k == zpb :
                    ac[c] += azp[c]
                    b[c] += 2.0*azp[c]*Vbound
    
    def sor(u):
        u0 = np.copy(u) # deep copy
        for i in range(xs, nx):
            for j in range(ys, ny):
                for k in range(zs, nz):
                    c = (i,j,k)
                    xp,xm,yp,ym,zp,zm = (i+1,j,k),(i-1,j,k),(i,j+1,k),(i,j-1,k),(i,j,k+1),(i,j,k-1)
                    
                    if i == xmb :
                        uxp ,uxm = u0[xp], zero
                    elif i == xpb :
                        uxp ,uxm = zero, u0[xm]
                    else:
                        uxp ,uxm = u0[xp], u0[xm]
                    
                    if j == ymb :
                        uyp ,uym = u0[yp], zero
                    elif j == ypb :
                        uyp ,uym = zero , u0[ym]
                    else:
                        uyp ,uym = u0[yp], u0[ym]
                        
                    if k == zmb :
                        uzp, uzm = u0[zp], zero
                    elif k == zpb :
                        uzp, uzm = zero, u0[zm]
                    else:
                        uzp ,uzm = u0[zp], u0[zm]
                    
                    ut = (axp[c]*uxp + axm[c]*uxm \
                        + ayp[c]*uyp + aym[c]*uym \
                        + azp[c]*uzp + azm[c]*uzm + b[c])/ac[c]
                    
                    u0[c] = ut
                    
        return u0
                    
    unew = np.copy(Vdir) # deep copy
    
    for _ in range(100):
        utmp = sor(unew)
        err = np.abs(unew - utmp).max() 
        unew = np.copy(utmp)
        if err < 10**(-3) : break
        
#     print(err)
        
    return unew, act

# calculate a pressure correction value

In [3]:
def caldp(p,u,v,w,acu,acv,acw,dx,dy,dz):
    
    zero = float(0.0)
    dp  = np.zeros_like(p)
    ac  = np.zeros_like(p)
    axp = np.zeros_like(p)
    axm = np.zeros_like(p)
    ayp = np.zeros_like(p)
    aym = np.zeros_like(p)
    azp = np.zeros_like(p)
    azm = np.zeros_like(p)
    b   = np.zeros_like(p)
    
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                c = (i, j, k)
                axp[c] =  zero if np.abs( acu[i+1,j,  k  ] ) < 10**(-8) else dy**2*dz**2/acu[i+1,j,  k  ] 
                axm[c] =  zero if np.abs( acu[i  ,j,  k  ] ) < 10**(-8) else dy**2*dz**2/acu[i  ,j,  k  ] 
                ayp[c] =  zero if np.abs( acv[i  ,j+1,k  ] ) < 10**(-8) else dx**2*dz**2/acv[i  ,j+1,k  ] 
                aym[c] =  zero if np.abs( acv[i  ,j,  k  ] ) < 10**(-8) else dx**2*dz**2/acv[i  ,j,  k  ] 
                azp[c] =  zero if np.abs( acw[i  ,j,  k+1] ) < 10**(-8) else dx**2*dy**2/acw[i  ,j,  k+1] 
                azm[c] =  zero if np.abs( acw[i  ,j,  k  ] ) < 10**(-8) else dx**2*dy**2/acw[i  ,j,  k  ] 
                
                ac[c] = axp[c]+axm[c]+ayp[c]+aym[c]+azp[c]+azm[c]
                
                b[c] = - (u[i+1,j,k] - u[i,j,k])*dy*dz \
                       - (v[i,j+1,k] - v[i,j,k])*dx*dz \
                       - (w[i,j,k+1] - w[i,j,k])*dx*dy
    
    def sor(dp0, omega):
        p0 = np.copy(dp0) # deep copy
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    c = (i,j,k)
                    xp,xm,yp,ym,zp,zm = (i+1,j,k),(i-1,j,k),(i,j+1,k),(i,j-1,k),(i,j,k+1),(i,j,k-1)
                    
                    if i == 0:
                        pxp ,pxm = p0[xp], zero
                    elif i == nx-1:
                        pxp ,pxm = zero, p0[xm]
                    else:
                        pxp ,pxm = p0[xp], p0[xm]
                    
                    if j == 0:
                        pyp ,pym = p0[yp], zero
                    elif j == ny-1:
                        pyp ,pym = zero , p0[ym]
                    else:
                        pyp ,pym = p0[yp], p0[ym]
                        
                    if k == 0:
                        pzp, pzm = p0[zp], zero
                    elif k == nz-1:
                        pzp, pzm = zero, p0[zm]
                    else:
                        pzp ,pzm = p0[zp], p0[zm]
                    
                    pt = (axp[c]*pxp + axm[c]*pxm \
                        + ayp[c]*pyp + aym[c]*pym \
                        + azp[c]*pzp + azm[c]*pzm + b[c])/ac[c]
                    
                    p0[c] += omega*(pt - p0[c])
    #                 p0[c] = ut
                    
        return p0
                    
    dpnew = np.copy(dp) # deep copy
    for nn in range(1000):
        dptmp = sor(dpnew, float(1.8))
        err =  np.abs(dpnew - dptmp).max() 
        dpnew = np.copy(dptmp)
        
        if err < 10**(-6) :
            print('SOR N =', nn)
            break
        
    return dpnew

# a convergent calculation of SIMPLE method

In [4]:
def SIMPLE(p,u,v,w,Uslip,Vslip,alphaU,alphaP,dx,dy,dz,nu):
    
    zero = float(0.0)
    nx, ny, nz = p.shape
    
    for nn in range(1000):
        
        u, acu = calupre(p,u,v,w,u,Uslip,alphaU,dx,dy,dz,nu,axis=0)
        v, acv = calupre(p,u,v,w,v,Vslip,alphaU,dx,dy,dz,nu,axis=1)
        w, acw = calupre(p,u,v,w,w,zero,alphaU,dx,dy,dz ,nu,axis=2)
        dp = caldp(p,u,v,w,acu,acv,acw,dx,dy,dz)
        
        p += alphaP*dp
        
        unew = np.copy(u) # deep copy
        vnew = np.copy(v) # deep copy
        wnew = np.copy(w) # deep copy
        
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    if i !=0 : unew[i,j,k] = u[i,j,k] + (dp[i-1,j,k] - dp[i,j,k])*dy*dz/acu[i,j,k]
                    if j !=0 : vnew[i,j,k] = v[i,j,k] + (dp[i,j-1,k] - dp[i,j,k])*dx*dz/acv[i,j,k]
                    if k !=0 : wnew[i,j,k] = w[i,j,k] + (dp[i,j,k-1] - dp[i,j,k])*dx*dy/acw[i,j,k]
        
        err =  max([np.abs(unew - u).max(), np.abs(vnew - v).max(), np.abs(wnew - w).max()])
        print('N', nn, 'error =', err)
        
        u = np.copy(unew) # deep copy
        v = np.copy(vnew) # deep copy
        w = np.copy(wnew) # deep copy
        
        if err < 10**(-6) : break
        
    return p,u,v,w

# main

In [5]:
%%time
dx,dy,dz = 0.05,0.05,0.05
nx,ny,nz = 20,20,20
nu = 0.01

alphaU = 0.5
alphaP = 0.8

Uslip = 1.0
Vslip = 0.0

p = np.zeros((nx,ny,nz))
u = np.zeros((nx+1,ny,nz))
v = np.zeros((nx,ny+1,nz))
w = np.zeros((nx,ny,nz+1))

p,u,v,w = SIMPLE(p,u,v,w,Uslip,Vslip,alphaU,alphaP,dx,dy,dz,nu)

SOR N = 125
N 0 error = 0.1054172791001945
SOR N = 120
N 1 error = 0.026840052366420453
SOR N = 109
N 2 error = 0.009562053929592945
SOR N = 106
N 3 error = 0.005360998254980088
SOR N = 93
N 4 error = 0.004474142497522898
SOR N = 94
N 5 error = 0.0032290649302523233
SOR N = 69
N 6 error = 0.002666631525062191
SOR N = 84
N 7 error = 0.002131719922800368
SOR N = 67
N 8 error = 0.001777679411929134
SOR N = 76
N 9 error = 0.0014729779128486165
SOR N = 60
N 10 error = 0.0012394939779675207
SOR N = 73
N 11 error = 0.0010233730467733015
SOR N = 63
N 12 error = 0.0009551915739345884
SOR N = 66
N 13 error = 0.0008101360144389114
SOR N = 64
N 14 error = 0.0007446578525629466
SOR N = 62
N 15 error = 0.0006630197513536884
SOR N = 60
N 16 error = 0.0006007713345940191
SOR N = 60
N 17 error = 0.0005408998700914824
SOR N = 60
N 18 error = 0.0004853426230619351
SOR N = 54
N 19 error = 0.00045843700257259916
SOR N = 60
N 20 error = 0.0003988421881952475
SOR N = 53
N 21 error = 0.00043021171823730275
SO

SOR N = 15
N 176 error = 3.0712207199357078e-06
SOR N = 16
N 177 error = 3.027014526146843e-06
SOR N = 15
N 178 error = 2.9520133110194635e-06
SOR N = 15
N 179 error = 2.9027338558917926e-06
SOR N = 15
N 180 error = 2.8461905576537827e-06
SOR N = 15
N 181 error = 2.790611308700619e-06
SOR N = 14
N 182 error = 2.7410191304755305e-06
SOR N = 15
N 183 error = 2.688334522854552e-06
SOR N = 14
N 184 error = 2.6389620582933926e-06
SOR N = 14
N 185 error = 2.577222071448171e-06
SOR N = 14
N 186 error = 2.530575231890486e-06
SOR N = 14
N 187 error = 2.4816696066465305e-06
SOR N = 13
N 188 error = 2.4391153694014456e-06
SOR N = 14
N 189 error = 2.3753895243561196e-06
SOR N = 13
N 190 error = 2.350730326894368e-06
SOR N = 13
N 191 error = 2.2904570768089716e-06
SOR N = 13
N 192 error = 2.250331456510324e-06
SOR N = 12
N 193 error = 2.2228703493720747e-06
SOR N = 13
N 194 error = 2.145835665184892e-06
SOR N = 12
N 195 error = 2.1459965736936315e-06
SOR N = 12
N 196 error = 2.074253453526742e-06
S

# export

In [6]:
import xarray as xr

In [7]:
uc = np.zeros((nx,ny,nz))
vc = np.zeros((nx,ny,nz))
wc = np.zeros((nx,ny,nz))

for i in range(nx) : uc[i,:,:] = u[i,:,:] + u[i+1,:,:]
for j in range(ny) : vc[:,j,:] = v[:,j,:] + v[:,j+1,:]
for k in range(nz) : wc[:,:,k] = w[:,:,k] + w[:,:,k+1]
    
dss = xr.Dataset( { 'p': (['x','y','z'], p), 'u': (['x','y','z'], uc), 'v': (['x','y','z'], vc), 'w': (['x','y','z'], wc) }
                  , coords={'x' : np.arange(nx)*dx + 0.5*dx , 'y' : np.arange(ny)*dy + 0.5*dy, 'z' : np.arange(nz)*dz + 0.5*dz }
                 )

dss.to_netcdf('output.nc')
dss.close()